In [1]:
# imports 
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt

# Data Set up

In [3]:
##### READ IN RAW DATA
print( "\nReading data from disk ...")
prop_raw = pd.read_csv('properties_2016.csv')
train_raw = pd.read_csv("train_2016_v2.csv")

# Processed data
# Full data set -> training + test
# training -> 90275 which includes logerror, transactiondate
# test -> prop + dummy logerror, transactiondate, 3064055 rows (excludes 11437 rows at the last where all features are nan)

df_full = pd.read_pickle("df_partial_filled_with_15.pkl") 
n_rows,n_columns = df_full.shape

train_w_prop_filled = df_full[0:90275]

# Prop df with filled data without the last 11437 empty rows
prop_filled_A = df_full.loc[90275::].drop(["logerror","transactiondate"],axis=1).reset_index()
prop_filled_A = prop_filled_A.drop(["index"],axis=1)

# prop df including 11437 empty rows
prop_filled_B = pd.concat([prop_filled_A,prop_raw.loc[2973780::]],ignore_index=True)

# Features with values missing more than threshold percentage will be removed or replaced with -1 or median
# Options
# 1. threshold removed w knn filled
# 2. threshold removed w meadian filled
# 3. threshold removed w -1 filled



Reading data from disk ...


C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def null_check(df):
    if df.isnull().sum().any() == True:
        print("NANs present")
    

def data_to_use(data,option,threshold=0.9):
    # Returns prop, train data given the options                                    
    na_perct = df_full.isnull().sum().values/(n_rows*1.0)
    df_na_summary = pd.DataFrame(df_full.columns.tolist(),columns=["Feature"])
    df_na_summary["% NA"] = na_perct
    active_features = df_na_summary["Feature"][df_na_summary["% NA"] <= threshold].values

    if data == "prop":
        if option == "threshold removed w knn filled":
            df_tmp = prop_filled_B[active_features] 
            df_tmp = df_tmp.fillna(prop_filled_B.median()) # filling the last empty rows
            null_check(df_tmp)
            
        if option == "threshold removed w median filled":
            df_tmp = prop_raw[active_features]
            df_tmp2 = prop_filled_B.copy()
            df_tmp.update(df_tmp2,join='left')
            df_tmp = df_tmp.fillna(prop_filled_B.median()) # filling the last empty rows
            null_check(df_tmp)
                       
        if option == "threshold removed w -1 filled":
            df_tmp = prop_raw[active_features]
            df_tmp2 = prop_filled_B.copy()
            df_tmp.update(df_tmp2,join='left')
            df_tmp = df_tmp.fillna(-1) # filling the last empty rows
            null_check(df_tmp)

                       
    if data == "train":
        if option == "threshold removed w knn filled":
            df_tmp = train_w_prop_filled[active_features]
            null_check(df_tmp)
                       
        if option == "threshold removed w median filled":
            df_tmp = train_raw[active_features]
            df_tmp = df_tmp.fillna(df_tmp.median())
            null_check(df_tmp)
                       
        if option == "threshold removed w -1 filled":
            df_tmp = train_raw[active_features]
            df_tmp = df_tmp.fillna(df_tmp.median())
            null_check(df_tmp)
                       
    return df_tmp


In [6]:
del prop_raw
del train_raw
del prop_filled_A
del df_full
gc.collect()

NameError: name 'prop_raw' is not defined

In [58]:
prop = pd.read_pickle("prop_del_me.pkl")
train = pd.read_pickle("train_del_me.pkl")

In [3]:
################
################
##  LightGBM changes ##
# V42 - sub_feature: 0.3 -> 0.35 : LB = 0.0643759
# V34 - sub_feature: 0.5 -> 0.42
# V33 - sub_feature: 0.5 -> 0.45 : LB = 0.0643866
# - sub_feature: 0.45 -> 0.3 : LB = 0.0643811 / 0.0643814 
################
################ 

# Parameters
XGB_WEIGHT = 0.6415
BASELINE_WEIGHT = 0.0056
OLS_WEIGHT = 0.0828

XGB1_WEIGHT = 0.8083  # Weight of first in combination of two XGB models

BASELINE_PRED = 0.0115   # Baseline based on mean of training data, per Oleg


In [40]:
################
################
##  LightGBM  ##
################
################

##### PROCESS DATA FOR LIGHTGBM

#prop = prop_filled_B
#train = train_w_prop_filled

print( "\nProcessing data for LightGBM ..." )
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

df_train = train        
        
#df_train = train.merge(df_props_filled, how='left', on='parcelid')
#df_train.fillna(df_train.median(),inplace = True)


x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1)
#x_train['Ratio_1'] = x_train['taxvaluedollarcnt']/x_train['taxamount']
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)


train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

x_train = x_train.values.astype(np.float32, copy=False)
d_train = lgb.Dataset(x_train, label=y_train)



Processing data for LightGBM ...
((1001, 53), (1001L,))


In [41]:
##### RUN LIGHTGBM
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.345    
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

print("\nFitting LightGBM model ...")
clf = lgb.train(params, d_train, 430)

del d_train; gc.collect()
del x_train; gc.collect()

print("\nPrepare for LightGBM prediction ...")
print("   Read sample file ...")
sample = pd.read_csv('sample_submission_del_me.csv')
print("   ...")
sample['parcelid'] = sample['ParcelId']
print("   Merge with property data ...")
df_test = sample.merge(prop, on='parcelid', how='left')
print("   ...")

del sample, prop; gc.collect()
print("   ...")
#df_test['Ratio_1'] = df_test['taxvaluedollarcnt']/df_test['taxamount']
x_test = df_test[train_columns]
print("   ...")
del df_test; gc.collect()
print("   Preparing x_test...")
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
print("   ...")
x_test = x_test.values.astype(np.float32, copy=False)
print("Test shape :", x_test.shape)

print("\nStart LightGBM prediction ...")
p_test = clf.predict(x_test)

del x_test; gc.collect()

print( "\nUnadjusted LightGBM predictions:" )
print( pd.DataFrame(p_test).head() )



Fitting LightGBM model ...

Prepare for LightGBM prediction ...
   Read sample file ...
   ...
   Merge with property data ...
   ...
   ...
   ...
   Preparing x_test...
   ...
('Test shape :', (1001L, 53L))

Start LightGBM prediction ...

Unadjusted LightGBM predictions:
          0
0  0.009339
1  0.009339
2  0.009339
3  0.009339
4  0.009339


In [43]:
################
################
##  XGBoost   ##
################
################

##### RE-READ PROPERTIES FILE
##### (I tried keeping a copy, but the program crashed.)

print( "\nRe-reading properties file ...")

properties = prop

##### PROCESS DATA FOR XGBOOST

train_df = train  


print( "\nProcessing data for XGBoost ...")
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))
    
for c in train_df.columns:
    train_df[c]=train_df[c].fillna(-1)
    if train_df[c].dtype == 'object' and c != "transactiondate":
        lbl = LabelEncoder()
        lbl.fit(list(train_df[c].values))
        train_df[c] = lbl.transform(list(train_df[c].values))
    
x_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
x_test = properties.drop(['parcelid'], axis=1)
# shape        
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))




# drop out ouliers
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]
x_train=train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

print('After removing outliers:')     
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))



Re-reading properties file ...

Processing data for XGBoost ...
Shape train: (1001, 57)
Shape test: (1001, 57)
After removing outliers:
Shape train: (983, 57)
Shape test: (1001, 57)


In [44]:
##### RUN XGBOOST
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': y_mean,
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

num_boost_rounds = 250
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
xgb_pred1 = model.predict(dtest)

print( "\nFirst XGBoost predictions:" )
print( pd.DataFrame(xgb_pred1).head() )



##### RUN XGBOOST AGAIN
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    'eta': 0.033,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

num_boost_rounds = 150
print("num_boost_rounds="+str(num_boost_rounds))

print( "\nTraining XGBoost again ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost again ...")
xgb_pred2 = model.predict(dtest)

print( "\nSecond XGBoost predictions:" )
print( pd.DataFrame(xgb_pred2).head() )



##### COMBINE XGBOOST RESULTS

xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2
#xgb_pred = xgb_pred1

print( "\nCombined XGBoost predictions:" )
print( pd.DataFrame(xgb_pred).head() )

del train_df
del x_train
del x_test
del properties
del dtest
del dtrain
del xgb_pred1
del xgb_pred2 
gc.collect()



Setting up data for XGBoost ...
num_boost_rounds=250

Training XGBoost ...

Predicting with XGBoost ...

First XGBoost predictions:
          0
0 -0.012415
1  0.007719
2  0.023997
3  0.027178
4  0.028186

Setting up data for XGBoost ...
num_boost_rounds=150

Training XGBoost again ...

Predicting with XGBoost again ...

Second XGBoost predictions:
          0
0 -0.017696
1 -0.035701
2  0.044129
3  0.009413
4  0.006101

Combined XGBoost predictions:
          0
0 -0.013427
1 -0.000604
2  0.027856
3  0.023772
4  0.023953


157

In [59]:
################
################
##    OLS     ##
################
################

# This section is derived from the1owl's notebook:
#    https://www.kaggle.com/the1owl/primer-for-the-zillow-pred-approach
# which I (Andy Harless) updated and made into a script:
#    https://www.kaggle.com/aharless/updated-script-version-of-the1owl-s-basic-ols
np.random.seed(17)
random.seed(17)

#train = pd.read_csv("train_2016_v3_filled.csv", parse_dates=["transactiondate"])
#train = train.drop(["Unnamed: 0"],axis=1)
#prop = pd.read_csv('properties_2016.csv')
#props_filled = df_full.loc[90275::].reset_index()
#props_filled = props_filled.drop(['logerror','transactiondate','index'],axis=1)
#prop.update(props_filled,join='left')

properties = prop


submission = pd.read_csv("sample_submission_del_me.csv")
print(len(train),len(properties),len(submission))

def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def MAE(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

#train = pd.merge(train, properties, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(submission, properties, how='left', left_on='ParcelId', right_on='parcelid')
properties = [] #memory

exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc] + ['transactiondate']

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])

reg = LinearRegression(n_jobs=-1)
reg.fit(train, y); print('fit...')
print(MAE(y, reg.predict(train)))
#train = [];  y = [] #memory

test_dates = ['2016-10-01','2016-11-01','2016-12-01','2017-10-01','2017-11-01','2017-12-01']
test_columns = ['201610','201611','201612','201710','201711','201712']



C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1001, 1001, 1001)
fit...
0.0651558014036


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [47]:
########################
########################
##  Combine and Save  ##
########################
########################

##### COMBINE PREDICTIONS
print( "\nCombining XGBoost, LightGBM, and baseline predicitons ..." )
lgb_weight = (1 - XGB_WEIGHT - BASELINE_WEIGHT) / (1 - OLS_WEIGHT)
xgb_weight0 = XGB_WEIGHT / (1 - OLS_WEIGHT)
baseline_weight0 =  BASELINE_WEIGHT / (1 - OLS_WEIGHT)
pred0 = xgb_weight0*xgb_pred + baseline_weight0*BASELINE_PRED + lgb_weight*p_test

print( "\nCombined XGB/LGB/baseline predictions:" )
print( pd.DataFrame(pred0).head() )

print( "\nPredicting with OLS and combining with XGB/LGB/baseline predicitons: ..." )
for i in range(len(test_dates)):
    test['transactiondate'] = test_dates[i]
    pred = OLS_WEIGHT*reg.predict(get_features(test)) + (1-OLS_WEIGHT)*pred0
    submission[test_columns[i]] = [float(format(x, '.4f')) for x in pred]
    print('predict...', i)

print( "\nCombined XGB/LGB/baseline/OLS predictions:" )
print( submission.head() )



##### WRITE THE RESULTS
from datetime import datetime
print( "\nWriting results to disk ..." )
submission.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)
print( "\nFinished ...")


Combining XGBoost, LightGBM, and baseline predicitons ...

Combined XGB/LGB/baseline predictions:
          0
0 -0.005727
1  0.003241
2  0.023147
3  0.020290
4  0.020416

Predicting with OLS and combining with XGB/LGB/baseline predicitons: ...
('predict...', 0)
('predict...', 1)
('predict...', 2)
('predict...', 3)
('predict...', 4)
('predict...', 5)

Combined XGB/LGB/baseline/OLS predictions:
   ParcelId  201610  201611  201612  201710  201711  201712
0  10754147 -0.0188 -0.0169 -0.0149 -0.0188 -0.0169 -0.0149
1  10759547 -0.0080 -0.0061 -0.0041 -0.0080 -0.0061 -0.0041
2  10843547  0.2054  0.2074  0.2093  0.2054  0.2074  0.2093
3  10859147 -0.0097 -0.0078 -0.0059 -0.0097 -0.0078 -0.0059
4  10879947 -0.0277 -0.0257 -0.0238 -0.0277 -0.0257 -0.0238

Writing results to disk ...

Finished ...
